In [12]:
import numpy as np
import pandas as pd
import edhec_risk_kit_206 as erk

from numpy.linalg import inv

In [56]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":]
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":]
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
rho_ = pd.DataFrame(ind49_rets[inds].corr(),
                    index=inds, columns=inds)
vols_ = (ind49_rets[inds].std()*np.sqrt(12))    
sigma_prior_ =  (vols_.T).dot(vols_) * rho_

ind49_ret.head(3)

,Hlth,Fin,Whlsl,Rtail,Food
2013-01,0.0966,0.1300,0.0646,0.0541,0.0754
2013-02,0.0078,0.0177,0.0207,0.0061,0.0483
2013-03,0.0591,0.0360,0.0325,0.0477,0.0798


In [127]:
w = ind49_mcap[inds].head(1).values.squeeze()
w = w / np.sum(w)
weights = pd.DataFrame(
    w,
    index = inds).T

weights

,Hlth,Fin,Whlsl,Rtail,Food
0,0.041663,0.175362,0.097411,0.546388,0.139176


In [97]:
def sample_cov(r, **kwargs):
    """
    Returns the sample covariance of the supplied returns
    """
    return r.cov()

def implied_returns(sigma, w, delta = 2.5):
    '''
    sigma:  VCV matrix
    w: portfolio weights
    delta: risk aversion coef(scalar)
    '''
    ir = delta * sigma.dot(w).squeeze()
    ir.name = 'Implied return'
    return ir

In [13]:
def proportional_prior(sigma, tau, p):
    '''
    Returns the He-Litterman simplified Omega, which is a 
        P x P matrix representing prior certainties
    Sigma: N x N Cov matrix
    tau: scalar
    p: K x N df linking Q and assets
    '''
    helit_omega = p.dot(tau*sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)), index = p.index, columns=p.index)

In [88]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

In [95]:
def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(sigma_prior,  w_prior, delta)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)


Q1. Which industry sector has the highest capweight

In [6]:
ind49_mcap.mean()

Hlth     0.005509
Fin      0.024322
Whlsl    0.013014
Rtail    0.070239
Food     0.017237
dtype: float64

Q2. Which industry sector has the highest implied return?  
Q3. Which industry sector has the lowest implied return?

In [128]:
IRs = implied_return(sigma_prior_, weights.T)
IRs

Hlth     0.152910
Fin      0.175580
Whlsl    0.201836
Rtail    0.224827
Food     0.158115
Name: Implied return, dtype: float64

Q4 - Q6. Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)

In [129]:
# relative view
q = pd.Series([
    0.03
    ]
)
p = pd.DataFrame([0.]*len(inds), index = inds).T
w_rtail = weights['Rtail'][0] / (weights['Rtail'][0] + weights['Whlsl'][0])
w_whlsl = weights['Whlsl'][0] / (weights['Rtail'][0] + weights['Whlsl'][0])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_rtail
p.iloc[0]['Whlsl'] = -w_whlsl
p*100

,Hlth,Fin,Whlsl,Rtail,Food
0,100.0,0.0,-15.130614,-84.869386,0.0


In [130]:
delta = 2.5
tau = 0.05
bl_mu, bl_sigma = bl(weights.T, sigma_prior_, p, q, delta = delta, tau = tau)
(bl_mu*100).round(1)

Hlth     18.0
Fin      16.9
Whlsl    19.3
Rtail    20.0
Food     14.5
dtype: float64

In [131]:
def w_star(delta, sigma, mu):
    return (inverse(sigma).dot(mu))/delta

w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)

Hlth     0.263700
Fin      0.167011
Whlsl    0.058876
Rtail    0.330244
Food     0.132549
dtype: float64

In [132]:
w_msr(bl_sigma, bl_mu)

Hlth     0.276885
Fin      0.175362
Whlsl    0.061820
Rtail    0.346756
Food     0.139176
dtype: float64

In [133]:
# relative view
q = pd.Series([
    0.05
    ]
)
bl_mu, bl_sigma = bl(weights.T, sigma_prior_, p, q, tau = 0.05)
(bl_mu*100).round(1)

Hlth     18.5
Fin      16.8
Whlsl    19.2
Rtail    19.5
Food     14.3
dtype: float64

In [107]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
(wstar*100).round(1)

Hlth     14.2
Fin       2.1
Whlsl    -0.9
Rtail    -5.1
Food      1.7
dtype: float64